In [11]:
import pandas as pd
import numpy as np

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

# Data Preparation

## Johns Hopkins GITHUB csv data


In [12]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

In [13]:
pd_raw.head()

Province/State Country/Region       Lat       Long  1/22/20  1/23/20  \
0            NaN    Afghanistan  33.93911  67.709953        0        0   
1            NaN        Albania  41.15330  20.168300        0        0   
2            NaN        Algeria  28.03390   1.659600        0        0   
3            NaN        Andorra  42.50630   1.521800        0        0   
4            NaN         Angola -11.20270  17.873900        0        0   

   1/24/20  1/25/20  1/26/20  1/27/20  ...  8/19/20  8/20/20  8/21/20  \
0        0        0        0        0  ...    37599    37856    37894   
1        0        0        0        0  ...     7812     7967     8119   
2        0        0        0        0  ...    39847    40258    40667   
3        0        0        0        0  ...     1024     1024     1045   
4        0        0        0        0  ...     2015     2044     2068   

   8/22/20  8/23/20  8/24/20  8/25/20  8/26/20  8/27/20  8/28/20  
0    37953    37999    38054    38070    38113    38129    38140  
1     8275     8427     8605     8759     8927     9083     9195  
2    41068    41460    41858    42228    42619    43016    43403  
3     1045     1045     1060     1060     1098     1098     1124  
4     2134     2171     2222     2283     2332     2415     2471  

[5 rows x 224 columns]

In [14]:
time_idx=pd_raw.columns[5:]

In [15]:
df_plot = pd.DataFrame({
    'date':time_idx})
df_plot.head()

date
0  1/23/20
1  1/24/20
2  1/25/20
3  1/26/20
4  1/27/20

In [16]:
pd_raw['Country/Region']

0                           Afghanistan
1                               Albania
2                               Algeria
3                               Andorra
4                                Angola
5                   Antigua and Barbuda
6                             Argentina
7                               Armenia
8                             Australia
9                             Australia
10                            Australia
11                            Australia
12                            Australia
13                            Australia
14                            Australia
15                            Australia
16                              Austria
17                           Azerbaijan
18                              Bahamas
19                              Bahrain
20                           Bangladesh
21                             Barbados
22                              Belarus
23                              Belgium
24                               Belize


In [17]:
pd_raw[pd_raw['Country/Region']=='India'].iloc[:,4::].sum(axis=0)[0:4]

1/22/20    0
1/23/20    0
1/24/20    0
1/25/20    0
dtype: int64

In [18]:
country_list=['Italy',
              'US',
              'Spain',
              'Germany',
              'Korea, South',
             ] 

In [ ]:
%matplotlib inline
df_plot.set_index('date').plot()

## Data Type Date

In [ ]:
df_plot.head()

In [ ]:
time_idx=[datetime.strptime( each,"%m/%d/%y") for each in df_plot.date] # convert to datetime
time_str=[each.strftime('%Y-%m-%d') for each in time_idx] # convert back to date ISO norm (str)

In [ ]:
df_plot['date']=time_idx
type(df_plot['date'][0])

In [ ]:
df_plot.head()

In [ ]:
df_plot.to_csv('../data/processed/COVID_small_flat_table.csv',sep=';',index=False)

## Relational data model - defining a primary key


In [ ]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)
pd_raw.head()

In [ ]:
pd_data_base=pd_raw.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})
pd_data_base['state']=pd_data_base['state'].fillna('no')  #ensure a sring, important for indexing

In [ ]:
pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)
pd_data_base.head()

In [ ]:
pd_relational_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )
pd_relational_model.head()


In [ ]:
pd_relational_model.dtypes

In [ ]:
pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')

In [ ]:
pd_relational_model.dtypes

In [ ]:
pd_relational_model.confirmed=pd_relational_model.confirmed.astype(int)

pd_relational_model.to_csv('../data/processed/COVID_relational_confirmed.csv',sep=';',index=False)

In [ ]:
print(pd_relational_model[pd_relational_model['country']=='US'].tail())

In [ ]:
data_path='../data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
pd_raw_US=pd.read_csv(data_path)
pd_raw_US.head()



In [ ]:
pd_raw_US=pd_raw_US.drop(['UID', 'iso2', 'iso3', 'code3', 'Country_Region','FIPS', 'Admin2',  'Lat', 'Long_', 'Combined_Key'],axis=1)
pd_data_base_US=pd_raw_US.rename(columns={'Province_State':'state'}).copy()


In [ ]:
pd_relational_model_US=pd_data_base_US.set_index(['state']) \
                                .T                              \
                                .stack()             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )
pd_relational_model_US['country']='US'
pd_relational_model_US['date']=[datetime.strptime( each,"%m/%d/%y") for each in pd_relational_model_US.date] # convert to datetime
pd_relational_model_US.head()

In [ ]:
pd_relational_model_all=pd_relational_model[pd_relational_model['country']!='US'].reset_index(drop=True)

In [ ]:
pd_relational_model_all=pd.concat([pd_relational_model_all,pd_relational_model_US],ignore_index=True)

In [ ]:
pd_relational_model_all.to_csv('../data/processed/20200424_COVID_relational_confirmed.csv',sep=';',index=False)

In [ ]:
print(pd_relational_model_all[pd_relational_model_all['country']=='US'])

### cross check on full sum US data

In [ ]:
check_US=pd_relational_model_all[pd_relational_model_all['country']=='US']

In [ ]:

check_US[['date','country','confirmed']].groupby(['date','country']).sum()